In [1]:
import sys
sys.path.append('/workspace/src/')

In [2]:
#get text chunks to index
from dotenv import dotenv_values, load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

import pickle
from tqdm import tqdm
import os
import pandas as pd

import langchain_core.documents
from langchain_community.retrievers import BM25Retriever
from database.model import Base, Document, Table
from database.chunk_model import Chunk_Base, Chunk

from preprocessing.utils import create_vectorstore, load_vectorstore, process_pickle_embeddings_in_chunks
import nltk
from nltk.tokenize import word_tokenize

nltk.download("punkt_tab")
db_vals = dotenv_values("/workspace/src/.env")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
chunk_engine = create_engine(f"postgresql+psycopg2://{db_vals['USER']}:{db_vals['PASSWORD']}@{db_vals['ADDRESS']}:{db_vals['PORT']}/cord19chunks", echo=False)
chunk_session = Session(chunk_engine)

In [24]:
#building vector store
vectorstore = create_vectorstore(chunks=[], save_path="vectorstore_small")

#vectorstore = load_vectorstore("/workspace/src/preprocessing/vectorstores/chromadb_store")

/workspace/src/preprocessing/utils.py:57: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model=embedding_model)


0it [00:00, ?it/s]

ValueError: Expected Embedings to be non-empty list or numpy array, got [] in upsert.

In [ ]:
process_pickle_embeddings_in_chunks(
    vectorstore=vectorstore,
    directory="/workspace/embeddings/embeddings/",
    chunk_session=chunk_session,
    chunk_size=100  # Adjust based on memory constraints and performance
)


In [ ]:
#building bm25 retriever/store
lang_docs = [langchain_core.documents.Document(page_content=chunk.chunk_text, metadata={"id": chunk.id, "doi": chunk.doi, "chunk_type": chunk.chunk_type}) for chunk in chunks]

In [8]:
table_chunks = chunk_session.query(Chunk).filter(Chunk.chunk_type == "RCTS_8192_1000").all()

In [10]:
passage_chunks = chunk_session.query(Chunk).filter(Chunk.chunk_type == "RCTS_512_100").all()

In [11]:
limit = 5000

In [29]:
chunks = table_chunks[:limit] + passage_chunks[:limit]

In [30]:
len(chunks)

10000

In [12]:
lang_table_docs = [langchain_core.documents.Document(page_content=chunk.chunk_text, metadata={"id": chunk.id, "doi": chunk.doi, "chunk_type": chunk.chunk_type}) for chunk in table_chunks][:limit]

In [13]:
lang_passage_docs = [langchain_core.documents.Document(page_content=chunk.chunk_text, metadata={"id": chunk.id, "doi": chunk.doi, "chunk_type": chunk.chunk_type}) for chunk in passage_chunks][:limit]

In [31]:
vectorstore = create_vectorstore(chunks=chunks, save_path="vectorstore_small")

  0%|          | 0/10 [00:00<?, ?it/s]

In [33]:
vectorstore.similarity_search("what is the origin of covid-19, table?")

  0%|          | 0/1 [00:00<?, ?it/s]

[Document(metadata={'chunk_type': 'RCTS_8192_1000', 'doi': '10.1016/j.dsx.2020.04.011#0', 'id': 8319191}, page_content="Table Name: Table 1 \nHeader: ['Feature', 'SARS-CoV-2', 'SARS-CoV-1', 'MERS-CoV'] \nContent: [['Origin', 'Wuhan, China', 'Guangdong, China', 'Jeddah, Saudi Arab'], ['Asymptomatic viral load', 'High', 'Less', 'Less'], ['Long period of infectivity', 'Yes', 'No', 'No'], ['Estimated R0', '2.2$_{e}$3.28', '2.0$_{e}$5.0', '< 1'], ['Median Incubation (days)', '6.4 (0$_{e}$24)', '4.6 (3.8$_{e}$5.8)', '5.2(1.9$_{e}$14.7)'], ['Serial interval (days)', '2.6$_{e}$7.5', '8.4', '12.6'], ['Case-fatality rate (%)', '3$_{e}$3.5', '9.6', '35.5'], ['Case-fatality rate with comorbidities (%)', '73.3', '46.0', '60.0'], ['Host', '', '', ''], ['Natural Host', 'Bats', 'Chinese horseshoe bats', 'Bats'], ['Intermediate Host', 'Pangolin', 'Civet', 'Camel'], ['Terminal Host', 'Humans', 'Humans', 'Humans'], ['Transmission', '', '', ''], ['Respiratory droplets', 'Yes', 'Yes', 'Yes'], ['Fomites, Co

In [17]:
retriever_bm25_table_small5k = BM25Retriever.from_documents(lang_table_docs, preprocess_func=word_tokenize, k=100)

Preprocessing texts


100%|██████████| 5000/5000 [00:00<00:00, 9542.10it/s]


In [18]:
retriever_bm25_passage_small5k = BM25Retriever.from_documents(lang_passage_docs, preprocess_func=word_tokenize, k=100)

Preprocessing texts


100%|██████████| 5000/5000 [00:00<00:00, 46019.34it/s]


In [21]:
retriever_bm25_passage_small5k.invoke("what is the origin of covid-19?")

[Document(metadata={'id': 1292810, 'doi': '10.1186/s43055-020-00245-8', 'chunk_type': 'RCTS_512_100'}, page_content='Title: CT chest of COVID-19 patients: what should a radiologist know? \n\n REVIEW\n\n<missing-text>\n\nCT chest of COVID-19 patients: what should a radiologist know?\n\nTamer F. Ali 1* , Mohamed A. Tawab 2 and Mona A ElHariri 1\n\nAbstract\n\nBackground: The aim of current work is to review the CT findings of COVID-19 in a pictorial study to help the radiologists to be familiar to imaging findings of COVID-19.'),
 Document(metadata={'id': 1292853, 'doi': '10.1186/s43055-020-00245-8', 'chunk_type': 'RCTS_512_100'}, page_content='15. Kanne JP. Chest CT findings in 2019 novel coronavirus (2019-nCoV) infections from Wuhan, China: key points for the radiologist. Radiology 2020.\n\n 16. Lee KS. Pneumonia associated with 2019 novel coronavirus: can computed tomographic findings help predict the prognosis of the disease? Korean J Radiol 2020.\n\n 17. Kim H. Outbreak of novel cor

In [22]:
pickle.dump(retriever_bm25_table_small5k, open("/workspace/src/preprocessing/vectorstores/bm25/retriever_bm25_table_small5k.pkl", "wb"))

In [23]:
pickle.dump(retriever_bm25_passage_small5k, open("/workspace/src/preprocessing/vectorstores/bm25/retriever_bm25_passage_small5k.pkl", "wb"))

In [102]:
#generate rank fusion for rannkigns

rankings = pickle.load(open("/workspace/src/data/rankings_new2.pkl", "rb"))

In [103]:
table_rankings = {key : value for key, value in rankings.items() if "table" in key}
passage_rankings = {key : value for key, value in rankings.items() if "passage" in key}

In [104]:
import copy
table_ranking_ids = copy.deepcopy(table_rankings)
passage_ranking_ids = copy.deepcopy(passage_rankings)

In [105]:
for query_var in table_ranking_ids.keys():
    for topic in table_ranking_ids[query_var].keys():
        table_ranking_ids[query_var][topic] = [doc.metadata['id'] for doc in table_ranking_ids[query_var][topic]]

for query_var in passage_ranking_ids.keys():
    for topic in passage_ranking_ids[query_var].keys():
        passage_ranking_ids[query_var][topic] = [doc.metadata['id'] for doc in passage_ranking_ids[query_var][topic]]

In [106]:
def reciprocal_rank_fusion(nested_dict, cutoff=200, k=60):
    """
    Applies Reciprocal Rank Fusion (RRF) to a set of ranked retrieval system outputs.
    
    Instead of applying the cutoff to each individual system’s ranking, this function 
    first computes fusion scores for all documents that appear in any system's ranking, 
    then sorts the fused results and applies the cutoff to the final ranking for each topic.
    
    Parameters:
      nested_dict (dict): A dictionary whose keys are retrieval system names and whose values
                          are dictionaries mapping topic IDs (as strings) to lists of document IDs.
                          For example:
                          {
                            'system1': {
                                '1': [doc1, doc2, doc3, ...],
                                '2': [doc7, doc8, ...],
                                ...
                            },
                            'system2': {
                                '1': [doc4, doc2, doc9, ...],
                                '2': [doc3, doc10, ...],
                                ...
                            },
                            ...
                          }
      cutoff (int): The number of top fused documents to return for each topic.
      k (int): The constant used in the RRF formula. For each appearance at rank r (1-indexed),
               the contribution is 1/(k + r).
               
    Returns:
      dict: A dictionary mapping each topic to its final fused ranking (a list of document IDs,
            sorted in descending order of their fusion scores), truncated to the top `cutoff` entries.
    """
    # Gather all topics present across all retrieval systems.
    topics = set()
    for system in nested_dict:
        topics.update(nested_dict[system].keys())
    
    # Dictionary to hold the final fused rankings per topic.
    fusion_results = {}
    
    # Process each topic individually.
    for topic in topics:
        doc_scores = {}  # Dictionary to accumulate scores for each document.
        
        # Iterate over each retrieval system.
        for system in nested_dict:
            if topic in nested_dict[system]:
                ranking = nested_dict[system][topic]
                # Iterate over the entire ranking from the system.
                for idx, doc in enumerate(ranking):
                    # Ranks are 1-indexed.
                    rank = idx + 1
                    # Calculate the RRF score for this document from the current system.
                    score = 1 / (k + rank)
                    doc_scores[doc] = doc_scores.get(doc, 0) + score
        
        # Sort documents by their cumulative RRF score (highest first).
        sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        
        # Apply the cutoff to the final fused ranking.
        fusion_results[topic] = [doc for doc, score in sorted_docs][:cutoff]
    
    return fusion_results

In [107]:
table_fusion = reciprocal_rank_fusion(table_ranking_ids)
passage_fusion = reciprocal_rank_fusion(passage_ranking_ids)

In [108]:
# safe the fusion results as json
import json
with open("/workspace/src/data/table_pool.json", "w") as f:
    json.dump(table_fusion, f)

with open("/workspace/src/data/passage_pool.json", "w") as f:
    json.dump(passage_fusion, f)


In [ ]:
passage_fusion["1"]

In [ ]:
passage_pool_for_rel["1"]

In [109]:
table_pool_for_rel = copy.deepcopy(table_fusion)
passage_pool_for_rel = copy.deepcopy(passage_fusion)
top_k = 10

for topic in table_pool_for_rel.keys():
    table_pool_for_rel[topic] = table_pool_for_rel[topic][:top_k]

for topic in passage_pool_for_rel.keys():
    passage_pool_for_rel[topic] = passage_pool_for_rel[topic][:top_k]

In [110]:
with open("/workspace/src/data/table_pool_for_rel.json", "w") as f:
    json.dump(table_pool_for_rel, f)

with open("/workspace/src/data/passage_pool_for_rel.json", "w") as f:
    json.dump(passage_pool_for_rel, f)


In [ ]:
len(passage_pool_for_rel)